## Klassifisering av landdekke

Denne demoen tar for seg klassifisering av landareale ved hjelp av et CNN (convolutional neural network).

Datasettet vi skal bruke er [EuroSAT](https://github.com/phelber/EuroSAT).

Som består av 27000 bilder av størrelse 64x64 piksler, fra satellitten Sentinel 2.

Disse bildene er delt inn i 10 klasser:

![](media/eurosat.jpg)

In [ ]:
# importer nødvendige pakker
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

### Etablere et datasett
EuroSAT ligger allerede inne i torchvision, noe som gjør at det er lett å eksperimentere med.
For egen data er dette steget en del mer jobb, men i dette tilfellet kan vi gjøre det med noen linjer kode.

Vi splitter datasettet i trenings- og test-sett.

In [ ]:
# trengs noen ganger om det oppstår ssl feil

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:

dataset = torchvision.datasets.EuroSAT("data",transform=torchvision.transforms.ToTensor(),download=True)

train,test = random_split(dataset,[int(len(dataset)*0.85),int(len(dataset)*0.15)])

batch_size = 8
trainloader = DataLoader(train,batch_size=batch_size,shuffle=True)
testloader = DataLoader(test,batch_size=batch_size,shuffle = False)

In [ ]:
dataset.classes

### Sette opp en modell

Vi skal teste noen froskjellige utgaver.

Først en veldig enkel modell vi lager selv. Så noen eksisterende modeller i kan importere med og uten pretrente vekter.

For en enkel modell setter vi opp en konvolusjonsnett med 5 lag.

For en ferdig modell velger vi modellen [MobileNetv3](https://arxiv.org/abs/1704.04861), som er et konvolusjonsnett for klassifisering med relativt god ytelse for ikke så mange parametere.
Også denne er lett tilgjengelig i torchvision.
Vi endrer på det siste laget for å endre antall klasser fra 1000 (ImageNet) til 10.

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,3)
        self.conv2 = nn.Conv2d(32,64,5)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(10816,10)
        

    def forward(self,x):
        x = self.relu(self.maxpool(self.conv1(x)))
        x = self.relu(self.maxpool(self.conv2(x)))
        x = torch.flatten(x,1)
        x = self.fc(x)

        return x

model = SimpleCNN()
if torch.cuda.is_available():
    model.to("cuda")


In [ ]:
# MOBILENET v3

# liten modell med tilfeldige vekter
# model = torchvision.models.mobilenet_v3_small() ; list(model.children())[-1][3] = torch.nn.Linear(1024,10)

# liten modell med pretrente vekter
# model = torchvision.models.mobilenet_v3_small(weights=torchvision.models.MobileNet_V3_Small_Weights) ; list(model.children())[-1][3] = torch.nn.Linear(1024,10)

# stor modell med pretrente vekter
# model = torchvision.models.mobilenet_v3_large(weights=torchvision.models.MobileNet_V3_Large_Weights) ; list(model.children())[-1][3] = torch.nn.Linear(1280,10)
if torch.cuda.is_available():
    model.to("cuda")


In [ ]:
import torchinfo

torchinfo.summary(model,(batch_size,3,64,64))

### Lage noen hjelpefunksjoner

#### En funksjon for å vise frem resultater frem resultater

In [ ]:
plt.rcParams["font.family"] = "monospace"
plt.rcParams["font.size"] = 8

def view_predictions(model,dataloader_iter):
    model.eval()
    batch = next(dataloader_iter)
    images,labels = batch
    with torch.no_grad():
        if torch.cuda.is_available():
            score = model(images.to("cuda")).to("cpu")
        else:
            score = model(images)
        vals, preds = torch.max(torch.softmax(score,axis=1),axis=1)

    fig, axs = plt.subplots(1, 1,figsize=(10,10),dpi=150,gridspec_kw = {'hspace':0})
    axs.imshow(torchvision.utils.make_grid(images,nrow=1).permute(2,1,0))
    axs.set_xticks([])
    axs.set_yticks([])
    axs.set_title(f"{'Labels:':8s} {dataset.classes[labels[0]]:20s} {dataset.classes[labels[1]]:25s} {dataset.classes[labels[2]]:25s} {dataset.classes[labels[3]]:25s} \n\
{'Preds:':8s} {dataset.classes[preds[0]]:20s} {dataset.classes[preds[1]]:25s} {dataset.classes[preds[2]]:25s} {dataset.classes[preds[3]]:25s}",loc="left")

#### Kjør funksjonen noen ganger for å se hva den predikerer

In [ ]:
view_predictions(model,iter(DataLoader(test,batch_size=4,shuffle=True)))

#### Teste hvor god modellen er

Vi lager en funksjon som tester kan kjøres på testsettet og gi ut to tall for å måle hvor god modellen er. Accuracy er et rent mål på hvor stor andel av bildene som blir riktig klassifisert git at man velger den klassen med høyest score.
Mean average precision (mAP) er et noe mer sofistikert mål som tar i betrakning hva slags score modellen gir for de ulike klassene.

In [ ]:
from sklearn.metrics import average_precision_score
def test_accuracy(model, dataloader,batch_lim=None):
    model.eval()
    map = []
    acc = []
    count = 0
    for batch in dataloader:
        images,labels = batch
        count +=1
        if batch_lim is not None:
            if count > batch_lim:
                break
        ap = []
        with torch.no_grad():
            if torch.cuda.is_available():
                score = model(images.to("cuda")).to("cpu")
            else:
                score = model(images)


            score = torch.softmax(score,dim=1)
            _, prediction = torch.max(score,dim=1)
        acc.append(np.array((prediction==labels).sum()/prediction.numel()))
        b,c = score.shape
        for j in range(c):
            class_label = (labels == j) * 1
            if torch.sum(class_label) == 0:
                continue
            ap.append(average_precision_score(class_label.flatten().numpy(),score[:,j].flatten().numpy()))
 

        map.append(np.mean(np.array(ap)))

    return np.mean(map), np.mean(acc)

In [ ]:
test_accuracy(model,testloader,batch_lim=None)

### Trening av modellen

In [ ]:
from torch.utils.tensorboard import SummaryWriter
lossfunc = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)
run = "runs/eurosat/SimpleCNN/"
writer = SummaryWriter(log_dir=run)
epochs = 5
running_loss = []
best = 0.3
for e in range(epochs):
    for i,batch in enumerate(trainloader):
        images,labels=batch
        model.train()
        optimizer.zero_grad()
        if torch.cuda.is_available():
            labels.to("cuda")
            score = model(images.to("cuda")).to("cpu")
        else:
            score = model(images)


        loss = lossfunc(score,labels)
        running_loss.append(loss.item())
        loss.backward()
        optimizer.step()

        if i == 0 or i % 500 == 499:
            m_loss = np.mean(running_loss)
            writer.add_scalar("loss",m_loss,e*len(trainloader) + i)
            map,acc = test_accuracy(model,testloader,batch_lim=500)
            writer.add_scalar("val/acc",acc,e*len(trainloader) + i)
            writer.add_scalar("val/mAP",map,e*len(trainloader) + i)
            print(f"E {e} B {i} Loss: {m_loss} mAP: {map*100:.2f}%   Acc: {acc*100:.2f}%")
            running_loss = []

            if map > best:
                torch.save(model.state_dict(),run+"best.pt")
                best = map
            



In [ ]:
model.load_state_dict(torch.load("runs/eurosat/SimpleCNN/best.pt"))

In [ ]:
import pandas as pd
import seaborn as sb
classes = dataset.classes
from sklearn.metrics import confusion_matrix
def create_confusion_matrix(model,testloader,batch_lim=None):
    cfms = []
    count = 0
    for i,batch in enumerate(testloader):
        images, labels = batch
        count +=1
        if batch_lim is not None:
            if count > batch_lim:
                break
        ap = []
        with torch.no_grad():
            if torch.cuda.is_available():
                score = model(images.to("cuda")).to("cpu")
            else:
                score = model(images)


            score = torch.softmax(score,dim=1)
            _, prediction = torch.max(score,dim=1)


            cfm = confusion_matrix(labels.flatten().numpy(),prediction.flatten().numpy(),normalize='true',labels=list(range(len(classes))))
            cfms.append(np.expand_dims(cfm,axis=0))

        if batch_lim is not None:
            if i >= batch_lim:
                break
    mean_cfm = np.mean(np.concatenate(cfms,axis=0),axis=0)
    df = pd.DataFrame(mean_cfm,index=[i for i in classes],columns=[i for i in classes])
    plt.figure(figsize=(15,15))
    cfm_plot = sb.heatmap(df,annot=True,cmap='viridis').get_figure()
    plt.savefig(run+"cfm.png")
    plt.show()

In [ ]:
create_confusion_matrix(model,testloader,batch_lim=None)